# Imports

In [35]:
import requests
import keyboard
import json
from ot2_api import OpentronsAPI
import numpy as np
import time
import cv2

# API commands

In [2]:
#Initialize the API.
openapi = OpentronsAPI()

In [3]:
#Toggle the lights on the robot. Can be used to check if the robot is connected.
openapi.toggle_lights()

In [4]:
#Home the robot.
openapi.home_robot()

Request status:
<Response [200]>
{
  "message": "Homing robot."
}


In [5]:
#Create a run.
openapi.create_run()

Request status:
<Response [201]>
{
  "data": {
    "id": "aa25a314-49f3-4051-9227-84763b9e937c",
    "ok": true,
    "createdAt": "2024-08-09T05:22:29.432713+00:00",
    "status": "idle",
    "current": true,
    "actions": [],
    "errors": [],
    "pipettes": [],
    "modules": [],
    "labware": [],
    "liquids": [],
    "labwareOffsets": [],
    "runTimeParameters": []
  }
}


In [6]:
#Load the pipette. Mount = "left" by default.
openapi.load_pipette()

Request status:
<Response [201]>
{
  "data": {
    "id": "a1485067-cd2c-446e-84f3-059e0c91d200",
    "createdAt": "2024-08-09T05:22:33.282368+00:00",
    "commandType": "loadPipette",
    "key": "a1485067-cd2c-446e-84f3-059e0c91d200",
    "status": "succeeded",
    "params": {
      "pipetteName": "p300_single_gen2",
      "mount": "left"
    },
    "result": {
      "pipetteId": "82b1a35b-3d94-4c9b-82a3-adc86511219a"
    },
    "startedAt": "2024-08-09T05:22:33.284545+00:00",
    "completedAt": "2024-08-09T05:22:35.331149+00:00",
    "intent": "setup",
    "notes": []
  }
}


In [7]:
#Gets basic info about the runs on the robot.
_ = openapi.get_run_info()

Total number of runs: 20
Current run ID: aa25a314-49f3-4051-9227-84763b9e937c
Current run status: idle


In [14]:
#Allows the user to delete a run.
runs_url = f"http://{openapi.ROBOT_IP}:31950/runs"
delete_run_url = f"{runs_url}/{openapi.run_id}"

r = requests.delete(
	url=delete_run_url,
	headers=openapi.HEADERS
	)

print(f"Request status:\n{r}\n{r.text}")

Request status:
<Response [200]>
{}


In [8]:
#Define a tip rack. This is the default tip rack for the robot.
TIP_RACK = "opentrons_96_tiprack_300ul"
#Load the tip rack. Slot = 1 by default.
openapi.load_labware(TIP_RACK, 1)

Labware ID:
0f11803a-6158-4047-9f4c-3d95461392cc



In [28]:
#Pick up a tip from the tip rack defined above. Tip well = "A1" by default.
openapi.pick_up_tip(openapi.labware_dct['1'], 'A1')

In [13]:
#Drop the tip back into the tip rack at the same position.
openapi.drop_tip(openapi.labware_dct['1'], 'A1')

In [98]:

class ManualRobotMovement:
    def __init__(self, openapi):
        self.openapi = openapi
        self.positions = []
        self.step = 1

        # Register the key bindings
        keyboard.on_press_key('up', lambda _: self.move_forward())
        keyboard.on_press_key('down', lambda _: self.move_backward())
        keyboard.on_press_key('left', lambda _: self.move_left())
        keyboard.on_press_key('right', lambda _: self.move_right())
        keyboard.on_press_key('+', lambda _: self.increase_step())
        keyboard.on_press_key('-', lambda _: self.decrease_step())
        keyboard.on_press_key('s', lambda _: self.save_position())
        keyboard.on_press_key('pagedown', lambda _: self.move_z_down())
        keyboard.on_press_key('pageup', lambda _: self.move_z_up())
        # Keep the program running until 'q' is pressed
        keyboard.wait('q')
        keyboard.unhook_all()

    def move_z_down(self):
        self.openapi.move_relative('z', -self.step)

    def move_z_up(self):
        self.openapi.move_relative('z', self.step)

    def move_forward(self):
        self.openapi.move_relative('y', self.step)

    def move_backward(self):
        self.openapi.move_relative('y', -self.step)

    def move_left(self):
        self.openapi.move_relative('x', -self.step)

    def move_right(self):
        self.openapi.move_relative('x', self.step)

    def increase_step(self):
        self.step *= 10
        print(f'Step: {self.step}')

    def decrease_step(self):
        self.step /= 10
        print(f'Step: {self.step}')

    def save_position(self):
        position = self.openapi.get_position()
        self.positions.append((position['x'], position['y'], position['z']))
        print(f"Saved position: {position}")


# Create an instance of the ManualRobotMovement class
manual_movement = ManualRobotMovement(openapi)

In [21]:
# Instruct the robot to move to a specific position using x,y,z coordinates. 
openapi.move_to_coordinates((55,220,100), min_z_height=1)

Request status:
<Response [201]>
{
  "data": {
    "id": "18cf2ef5-3b25-4218-b9e8-56856f321696",
    "createdAt": "2024-08-09T05:46:47.495500+00:00",
    "commandType": "moveToCoordinates",
    "key": "18cf2ef5-3b25-4218-b9e8-56856f321696",
    "status": "succeeded",
    "params": {
      "minimumZHeight": 1.0,
      "forceDirect": false,
      "pipetteId": "82b1a35b-3d94-4c9b-82a3-adc86511219a",
      "coordinates": {
        "x": 55.0,
        "y": 220.0,
        "z": 100.0
      }
    },
    "result": {
      "position": {
        "x": 55.0,
        "y": 220.0,
        "z": 100.0
      }
    },
    "startedAt": "2024-08-09T05:46:47.497772+00:00",
    "completedAt": "2024-08-09T05:46:48.403448+00:00",
    "intent": "setup",
    "notes": []
  }
}


In [100]:
np.save('marker_positions.npy', np.array(manual_movement.positions) )

In [29]:
tf_mtx = np.load('./tf_mtx.npy')
cX, cY = (1311, 925)
X, Y, _ = tf_mtx @ (cX, cY, 1)

In [27]:
X,Y

(33.78384420804972, 341.04063282572406)

In [34]:
openapi.move_to_coordinates((X,Y,100), min_z_height=1)


Request status:
<Response [201]>
{
  "data": {
    "id": "d312e75a-6323-4099-825d-102883dbc501",
    "createdAt": "2024-08-09T06:39:57.749568+00:00",
    "commandType": "moveToCoordinates",
    "key": "d312e75a-6323-4099-825d-102883dbc501",
    "status": "succeeded",
    "params": {
      "minimumZHeight": 1.0,
      "forceDirect": false,
      "pipetteId": "82b1a35b-3d94-4c9b-82a3-adc86511219a",
      "coordinates": {
        "x": 63.838585693586815,
        "y": 316.11564044403696,
        "z": 100.0
      }
    },
    "result": {
      "position": {
        "x": 63.838585693586815,
        "y": 316.11564044403696,
        "z": 100.0
      }
    },
    "startedAt": "2024-08-09T06:39:57.751423+00:00",
    "completedAt": "2024-08-09T06:39:58.334620+00:00",
    "intent": "setup",
    "notes": []
  }
}


In [89]:
coordinates = openapi.get_position()

{'x': 33.00000000000001, 'y': 301.0, 'z': 50.0}


In [14]:
markers = np.load('marker_positions.npy')

In [18]:
markers

array([[ 33.7, 341.1,   1. ],
       [ 85. , 340.2,   1. ],
       [ 86.5, 304.2,   1. ],
       [ 33.8, 304. ,   1. ]])

### Load and execute protocol

In [7]:
PROTOCOL_FILE = "./protocols/test.py"
LABWARE_FILE = "./vwr_96_tiprack_200ul.json"
openapi.upload_protocol(PROTOCOL_FILE, LABWARE_FILE)

FileNotFoundError: [Errno 2] No such file or directory: './protocols/test.py'

In [28]:
openapi.create_run(protocol_id=openapi.protocol_id)

Request status:
<Response [201]>
{
  "data": {
    "id": "f6a256ac-261c-41f6-ba4e-0692bf2055c4",
    "ok": true,
    "createdAt": "2024-07-09T23:42:14.581162+00:00",
    "status": "idle",
    "current": true,
    "actions": [],
    "errors": [],
    "pipettes": [],
    "modules": [],
    "labware": [],
    "liquids": [],
    "labwareOffsets": [],
    "runTimeParameters": [],
    "protocolId": "5027fe50-5f0a-423c-9ce4-2b2d8e18e3df"
  }
}


In [29]:
openapi.run_protocol()

In [22]:
protocols_url = f"http://{openapi.ROBOT_IP}:31950/protocols"

r = requests.get(
	url=protocols_url,
	headers=openapi.HEADERS
	)

protocols_dict = json.loads(r.text)
print("Protocols list:")
for p in protocols_dict["data"]:
	print(p)

Protocols list:
{'id': 'dafb7d77-8245-448d-9864-27fa1e3137d5', 'createdAt': '2024-05-14T22:57:28.389017+00:00', 'files': [{'name': 'test_vwr_96_tiprack_200ul.py', 'role': 'main'}, {'name': 'vwr_96_tiprack_200ul.json', 'role': 'labware'}], 'protocolType': 'python', 'robotType': 'OT-2 Standard', 'metadata': {'apiLevel': '2.0'}, 'analyses': [], 'analysisSummaries': [{'id': 'a9ee0d6c-94d5-4001-894e-e31f6a51450c', 'status': 'completed'}], 'key': 'a90763dc-bb0f-4f8b-849d-3e5b31cf6a46'}
{'id': 'e40e72fa-9362-4434-adcc-b72c6c67ed71', 'createdAt': '2024-05-14T23:13:32.939907+00:00', 'files': [{'name': 'test_vwr_96_tiprack_200ul.py', 'role': 'main'}, {'name': 'vwr_96_tiprack_200ul.json', 'role': 'labware'}], 'protocolType': 'python', 'robotType': 'OT-2 Standard', 'metadata': {'apiLevel': '2.0'}, 'analyses': [], 'analysisSummaries': [{'id': '24db9ff8-f373-48cd-807b-9a07a89882e4', 'status': 'completed'}], 'key': '918a57f2-1e17-4735-af45-cd1bcad0aef9'}
{'id': '3047424e-0c54-45e0-8783-1444f0e75e26',

# Drafts

### Liquid pickup attempt

In [36]:
TIP_RACK = "opentrons_96_tiprack_300ul"
TIP_RACK2 = "vwr_96_tiprack_200ul"
RESERVOIR = "nest_12_reservoir_15ml"
WELL_PLATE = "corning_96_wellplate_360ul_flat"

In [37]:

LABWARE_FILE = "./vwr_96_tiprack_200ul.json"

HEADERS = {"opentrons-version": "3"}

protocols_url = f"http://{openapi.ROBOT_IP}:31950/protocols"

labware_file_payload = open(LABWARE_FILE, "rb")

r = requests.post(
	url=protocols_url,
	headers=HEADERS,
	files=[("files", labware_file_payload)]
	)

r_dict = json.loads(r.text)

labware_file_payload.close()

FileNotFoundError: [Errno 2] No such file or directory: './vwr_96_tiprack_200ul.json'

In [48]:
command_dict = {
	"data": {
		"commandType": "aspirateInPlace",
		"params": {
			"flowRate": 50,
			"volume": 100,
			"pipetteId": openapi.pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	data=command_payload,
    params={"waitUntilComplete": True}
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "aspirateInPlace", "params": {"flowRate": 50, "volume": 100, "pipetteId": "e649fddd-1ac5-4d61-ad58-b460d43c89aa"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "070e3e10-8ab8-49f7-a9b1-653f67f51bbc", "createdAt": "2024-08-09T04:41:46.398525+00:00", "commandType": "aspirateInPlace", "key": "070e3e10-8ab8-49f7-a9b1-653f67f51bbc", "status": "succeeded", "params": {"flowRate": 50.0, "volume": 100.0, "pipetteId": "e649fddd-1ac5-4d61-ad58-b460d43c89aa"}, "result": {"volume": 100.0}, "startedAt": "2024-08-09T04:41:46.400443+00:00", "completedAt": "2024-08-09T04:41:48.781001+00:00", "intent": "setup", "notes": []}}



In [29]:
# openapi.move_to_coordinates((200,50,50), min_z_height=1)
# openapi.move_to_coordinates((200,50,5), min_z_height=1)


command_dict = {
	"data": {
		"commandType": "aspirate",
		"params": {
			"labwareId": labware_3_id,
			"wellName": "A1",
			"wellLocation": {
				"origin": "top", "offset": {"x": 0, "y": 0, "z": -40}
			},
			"flowRate": 10,
			"volume": 100,
			"pipetteId": openapi.pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	data=command_payload,
    params={"waitUntilComplete": True}
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "aspirate", "params": {"labwareId": "8c039e2a-5bd8-40d1-bd3b-3c06ea571ac9", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 0, "y": 0, "z": -40}}, "flowRate": 10, "volume": 100, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "b24f6040-137d-4a03-9e4f-84e973fb1c13", "createdAt": "2024-06-14T07:13:52.252198+00:00", "commandType": "aspirate", "key": "b24f6040-137d-4a03-9e4f-84e973fb1c13", "status": "succeeded", "params": {"labwareId": "8c039e2a-5bd8-40d1-bd3b-3c06ea571ac9", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 0.0, "y": 0.0, "z": -40.0}}, "flowRate": 10.0, "volume": 100.0, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "result": {"position": {"x": 196.38, "y": 42.74, "z": 4.399999999999997}, "volume": 100.0}, "startedAt": "2024-06-14T07:13:52.253813+00:00", "completedAt": "2024-06-14T07:14:06.759121+00:00", "intent": "setup"}}



In [30]:
command_dict = {
	"data": {
		"commandType": "dispense",
		"params": {
			"labwareId": labware_2_id,
			"wellName": "A2",
			"wellLocation": {
				"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}
			},
			"flowRate": 10,
			"volume": 100,
			"pipetteId": openapi.pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	data=command_payload,
    params={"waitUntilComplete": True}
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "dispense", "params": {"labwareId": "b7bc5044-bf0e-40f0-afc5-b4d10a037596", "wellName": "A2", "wellLocation": {"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}}, "flowRate": 10, "volume": 100, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "4386471b-66b2-4395-a355-a151021e5adc", "createdAt": "2024-06-14T07:14:06.843683+00:00", "commandType": "dispense", "key": "4386471b-66b2-4395-a355-a151021e5adc", "status": "succeeded", "params": {"labwareId": "b7bc5044-bf0e-40f0-afc5-b4d10a037596", "wellName": "A2", "wellLocation": {"origin": "top", "offset": {"x": 0.0, "y": 0.0, "z": 0.0}}, "flowRate": 10.0, "volume": 100.0, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "result": {"position": {"x": 288.38, "y": 74.24, "z": 14.219999999999999}, "volume": 100.0}, "startedAt": "2024-06-14T07:14:06.845252+00:00", "completedAt": "2024-06-14T07:14:19.409160+00:00", "intent": "setup"}}



In [31]:
command_dict = {
	"data": {
		"commandType": "blowout",
		"params": {
			"labwareId": labware_2_id,
			"wellName": "A2",
			"wellLocation": {
				"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}
			},
			"flowRate": 10,
			"pipetteId": openapi.pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	data=command_payload,
    params={"waitUntilComplete": True}
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "blowout", "params": {"labwareId": "b7bc5044-bf0e-40f0-afc5-b4d10a037596", "wellName": "A2", "wellLocation": {"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}}, "flowRate": 10, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "25f080f9-bf07-4ea1-831f-f2e6c9e661f3", "createdAt": "2024-06-14T07:14:19.467575+00:00", "commandType": "blowout", "key": "25f080f9-bf07-4ea1-831f-f2e6c9e661f3", "status": "succeeded", "params": {"labwareId": "b7bc5044-bf0e-40f0-afc5-b4d10a037596", "wellName": "A2", "wellLocation": {"origin": "top", "offset": {"x": 0.0, "y": 0.0, "z": 0.0}}, "flowRate": 10.0, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "result": {"position": {"x": 288.38, "y": 74.24, "z": 14.219999999999999}}, "startedAt": "2024-06-14T07:14:19.469225+00:00", "completedAt": "2024-06-14T07:14:23.883102+00:00", "intent": "setup"}}



In [32]:
command_dict = {
	"data": {
		"commandType": "dropTip",
		"params": {
			"labwareId": labware_1_id,
			"wellName": "A1",
			"wellLocation": {
				"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}
			},
			"pipetteId": openapi.pipette_id
		},
		"intent": "setup"
	}
}

command_payload = json.dumps(command_dict)
print(f"Command:\n{command_payload}\n")

r = requests.post(
	url=openapi.commands_url,
	headers=openapi.HEADERS,
	data=command_payload,
    params={"waitUntilComplete": True}
	)

print(f"Response:\n{r}\n{r.text}\n")

Command:
{"data": {"commandType": "dropTip", "params": {"labwareId": "35649278-ee0c-412e-8a6b-c1e6f6144d68", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 0, "y": 0, "z": 0}}, "pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "3e0af3fb-2557-470a-87c7-f63c4822a335", "createdAt": "2024-06-14T07:30:30.261117+00:00", "commandType": "dropTip", "key": "3e0af3fb-2557-470a-87c7-f63c4822a335", "status": "succeeded", "params": {"pipetteId": "e2534277-8a57-4760-8ea8-aebbdf19aa17", "labwareId": "35649278-ee0c-412e-8a6b-c1e6f6144d68", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 0.0, "y": 0.0, "z": 0.0}}, "alternateDropLocation": false}, "result": {"position": {"x": 14.38, "y": 74.24, "z": 64.69}}, "startedAt": "2024-06-14T07:30:30.262871+00:00", "completedAt": "2024-06-14T07:30:43.309438+00:00", "intent": "setup"}}

